Abdullah Itani
Abdullatif Itani

# Given the data in the excel, we wanted to try and predict the starting age for smoking depending on some of the given variables namely:
I see myself as someone who is extraverted, enthusiastic:	I see myself as someone who is critical, quarrelsome:	I see myself as someone who is dependable, self-disciplined:	I see myself as someone who is anxious, easily upset:	I see myself as someone who is open to new experiences:	I see myself as someone who is reserved, quiet:	I see myself as someone who is sympathetic, warm:	I see myself as someone who is disorganized, careless:	I see myself as someone who is calm, emotionally stable:	I see myself as someone who is conventional, uncreative:

These are columns 3-12 included, result is on 19 (age) we decided to use Random Forest


In [ ]:
!pip install pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
def build_model(file_path):
    # Load the dataset from the provided Excel file
    df = pd.read_excel(file_path)

    # Select columns 3-12 as features (assuming 0-based indexing)
    features = df.iloc[:, 4:14]  # Changed from 4:13 to 3:13
    target = df.iloc[:, 20]      # Column 21 (age of first cigarette)

    # Define the mapping for Likert scale responses
    likert_mapping = {
        'Disagree strongly': 1,
        'Disagree moderately': 2,
        'Disagree a little': 3,
        'Neither agree nor disagree': 4,
        'Agree a little': 5,
        'Agree moderately': 6,
        'Agree strongly': 7
    }

    # Clean the features: strip whitespace, standardize text, and apply mapping
    features = features.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    features = features.replace(likert_mapping)

    # Handle missing values by filling with the median of each column
    features.fillna(features.median(), inplace=True)

    # Check for any remaining non-numeric values
    if features.isnull().values.any():
        print("Warning: There are still unmapped or invalid values in the features.")
        print(features[features.isnull().any(axis=1)])
        return

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                        test_size=0.2, random_state=42)

    # Initialize a Random Forest Regressor
    model = RandomForestRegressor(random_state=42, n_estimators=100)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print evaluation metrics
    print("Model Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with the actual file path

# Build and train the model
if __name__ == "__main__":
    model = build_model(file_path)

<ipython-input-26-c503af5eca05>:21: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  features = features.applymap(lambda x: x.strip() if isinstance(x, str) else x)
<ipython-input-26-c503af5eca05>:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace(likert_mapping)


Model Evaluation Metrics:
Mean Absolute Error (MAE): 21.27
Mean Squared Error (MSE): 12055.67
R^2 Score: -1002.46


In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Update file path
df = pd.read_excel(file_path)

# Select features and target
features = df.iloc[:, 4:14]  # Columns 5-13 (Likert-scale personality traits)
target = df.iloc[:, 20]      # Column 21 (Age first cigarette)

# Map Likert responses to numerical values
likert_mapping = {
    'Disagree strongly': 1,
    'Disagree moderately': 2,
    'Disagree a little': 3,
    'Neither agree nor disagree': 4,
    'Agree a little': 5,
    'Agree moderately': 6,
    'Agree strongly': 7
}

# Apply mapping and handle missing data
features = features.replace(likert_mapping)
features.fillna(features.median(), inplace=True)

# Check correlation with the target variable
correlation = features.corrwith(target)
print("Correlation with Target (Sorted):")
print(correlation.sort_values())


Correlation with Target (Sorted):
I see myself as someone who is anxious, easily upset:          -0.131818
I see myself as someone who is disorganized, careless:         -0.094892
I see myself as someone who is conventional, uncreative:       -0.073528
I see myself as someone who is critical, quarrelsome:          -0.017418
I see myself as someone who is extraverted, enthusiastic:       0.032044
I see myself as someone who is open to new experiences:         0.064728
I see myself as someone who is dependable, self-disciplined:    0.071184
I see myself as someone who is sympathetic, warm:               0.084954
I see myself as someone who is calm, emotionally stable:        0.100191
I see myself as someone who is reserved, quiet:                 0.125105
dtype: float64


<ipython-input-27-828c032a2097>:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace(likert_mapping)


Model Evaluation Metrics:
*   Mean Absolute Error (MAE): 21.27
*   Mean Squared Error (MSE): 12055.67
*   R^2 Score: -1002.46

Our metrics are very bad so lets try to use less features that correlate more with the result

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def build_model(file_path):
    # Load the dataset from the provided Excel file
    df = pd.read_excel(file_path)

    # Select columns 12, 10, and 9 as features (assuming 0-based indexing)
    features = df.iloc[:, [12, 10, 9]]  # Use specific columns as features
    target = df.iloc[:, 20]             # Column 21 (age of first cigarette)

    # Define the mapping for Likert scale responses
    likert_mapping = {
        'Disagree strongly': 1,
        'Disagree moderately': 2,
        'Disagree a little': 3,
        'Neither agree nor disagree': 4,
        'Agree a little': 5,
        'Agree moderately': 6,
        'Agree strongly': 7
    }

    # Apply mapping and handle missing values
    features = features.replace(likert_mapping)
    features.fillna(features.median(), inplace=True)

    # Check for any remaining non-numeric values
    if features.isnull().values.any():
        print("Warning: There are still unmapped or invalid values in the features.")
        print(features[features.isnull().any(axis=1)])
        return

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                        test_size=0.2, random_state=42)

    # Initialize a Random Forest Regressor
    model = RandomForestRegressor(random_state=42, n_estimators=100)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print evaluation metrics
    print("Model Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with the actual file path

# Build and train the model
if __name__ == "__main__":
    model = build_model(file_path)


Model Evaluation Metrics:
Mean Absolute Error (MAE): 10.92
Mean Squared Error (MSE): 2640.79
R^2 Score: -218.81


<ipython-input-28-14c6c3281de8>:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace(likert_mapping)


In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Update file path
df = pd.read_excel(file_path)

# Select columns 12, 10, 9 as features and column 21 as the target
features = df.iloc[:, [12, 10, 9]]
target = df.iloc[:, 20]  # Column 21 (age of first cigarette)

# Define the mapping for Likert scale responses
likert_mapping = {
    'Disagree strongly': 1,
    'Disagree moderately': 2,
    'Disagree a little': 3,
    'Neither agree nor disagree': 4,
    'Agree a little': 5,
    'Agree moderately': 6,
    'Agree strongly': 7
}

# Map Likert responses to numerical values
features = features.replace(likert_mapping)

# Handle missing values
features.fillna(features.median(), inplace=True)

# Compute correlations with the target variable
correlation = features.corrwith(target)
print("Correlation with Target (Sorted):")
print(correlation.sort_values())


Correlation with Target (Sorted):
I see myself as someone who is sympathetic, warm:           0.084954
I see myself as someone who is calm, emotionally stable:    0.100191
I see myself as someone who is reserved, quiet:             0.125105
dtype: float64


<ipython-input-29-17f637d482c2>:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace(likert_mapping)


# Again we have horrendous results
Model Evaluation Metrics:
*   Mean Absolute Error (MAE): 10.92
*   Mean Squared Error (MSE): 2640.79
*   R^2 Score: -218.81

Looking through the data we saw that there was an outlier age of 2016 and 189, and the min max of the data was 13, 24. Hence we decided to make the model only consider ages below 30, and make a prediction between ages 10-30

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def build_model(file_path):
    # Load the dataset from the provided Excel file
    df = pd.read_excel(file_path)

    # Select columns 12, 10, and 9 as features (assuming 0-based indexing)
    features = df.iloc[:, [12, 10, 9]]  # Use specific columns as features
    target = df.iloc[:, 20]             # Column 21 (age of first cigarette)

    # Remove rows where the target age is above 30
    valid_indices = target <= 30
    features = features[valid_indices]
    target = target[valid_indices]

    # Define the mapping for Likert scale responses
    likert_mapping = {
        'Disagree strongly': 1,
        'Disagree moderately': 2,
        'Disagree a little': 3,
        'Neither agree nor disagree': 4,
        'Agree a little': 5,
        'Agree moderately': 6,
        'Agree strongly': 7
    }

    # Apply mapping and handle missing values
    features = features.replace(likert_mapping)
    features.fillna(features.median(), inplace=True)

    # Check for any remaining non-numeric values
    if features.isnull().values.any():
        print("Warning: There are still unmapped or invalid values in the features.")
        print(features[features.isnull().any(axis=1)])
        return

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                        test_size=0.2, random_state=42)

    # Initialize a Random Forest Regressor
    model = RandomForestRegressor(random_state=42, n_estimators=100)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Constrain predictions to the range [10, 30]
    y_pred = y_pred.clip(10, 30)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print evaluation metrics
    print("Model Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"R^2 Score: {r2:.2f}")

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with the actual file path

# Build and train the model
if __name__ == "__main__":
    model = build_model(file_path)


<ipython-input-30-0f2fd291bb36>:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace(likert_mapping)


Model Evaluation Metrics:
Mean Absolute Error (MAE): 2.03
Mean Squared Error (MSE): 7.45
R^2 Score: 0.09


In conclusion we managed to improve our predictions as much as possible but in the end we can deduce that the given variables aren't very suitable to predict the age a person smoked a full cigarette, so lets try something else!

# Second Idea
As we was browsing through the data we were interested in the subject of cigarette brands however the data is way to jumbled to get an accurate prediction on what cigarette brand someone is using, so we decided to see if someone can afford his favorite cigarette brand.

#Affording favorite cigarette brand
The data I will use here and might change later on are: Employment status(47 AV), How often do you feel stressed?(46 AU), To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?(43 AR), How would you describe your current income sufficiency?(41 AP), What is the highest level of education you have attained?(29 AD)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

def logistic_regression_model(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Select features and target
    features = df[[
        "Employment Status",
        "How often do you feel stressed?",
        "To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?",
        "How would you describe your current income sufficiency?",
        "What is the highest level of education you have attained?"
    ]]
    target = df.iloc[:, 24]  # Column 24: Can afford favorite cigarette brand?

    # Encode categorical variables
    categorical_features = features.select_dtypes(include=['object']).columns
    features = pd.get_dummies(features, columns=categorical_features, drop_first=True)  # One-hot encoding for categorical variables

    # Encode the target variable (1 for Yes, 0 for No)
    target = target.map({"Yes": 1, "No": 0})

    # Handle missing values
    features.fillna(features.median(), inplace=True)
    target.dropna(inplace=True)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Initialize the Logistic Regression model
    model = LogisticRegression(random_state=42, max_iter=1000)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (Yes)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    print("Logistic Regression Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with your file path

# Build and train the model
if __name__ == "__main__":
    model = logistic_regression_model(file_path)


Logistic Regression Model Evaluation:
Accuracy: 0.67
Precision: 0.74
Recall: 0.83
F1 Score: 0.78
ROC-AUC Score: 0.69

Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.31      0.36        13
           1       0.74      0.83      0.78        30

    accuracy                           0.67        43
   macro avg       0.59      0.57      0.57        43
weighted avg       0.65      0.67      0.65        43



We tried using Logistic regression but the results weren't very accurate, and the predictions of "Yes" were more accurate than "No" due to the imbalance in the data set, lets try random forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

def random_forest_model(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Select features and target
    features = df[[
        "Employment Status",
        "How often do you feel stressed?",
        "To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?",
        "How would you describe your current income sufficiency?",
        "What is the highest level of education you have attained?"
    ]]
    target = df.iloc[:, 24]  # Column 24: Can afford favorite cigarette brand?

    # Encode categorical variables
    categorical_features = features.select_dtypes(include=['object']).columns
    features = pd.get_dummies(features, columns=categorical_features, drop_first=True)  # One-hot encoding for categorical variables

    # Encode the target variable (1 for Yes, 0 for No)
    target = target.map({"Yes": 1, "No": 0})

    # Handle missing values
    features.fillna(features.median(), inplace=True)
    target.dropna(inplace=True)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Initialize the Random Forest Classifier
    model = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (Yes)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    print("Random Forest Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with your file path

# Build and train the model
if __name__ == "__main__":
    model = random_forest_model(file_path)


Random Forest Model Evaluation:
Accuracy: 0.70
Precision: 0.81
Recall: 0.73
F1 Score: 0.77
ROC-AUC Score: 0.77

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.62      0.55        13
           1       0.81      0.73      0.77        30

    accuracy                           0.70        43
   macro avg       0.66      0.67      0.66        43
weighted avg       0.72      0.70      0.71        43



In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def feature_correlation_analysis(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Select features and target
    features = df[[
        "Employment Status",
        "How often do you feel stressed?",
        "To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?",
        "How would you describe your current income sufficiency?",
        "What is the highest level of education you have attained?"
    ]]
    target = df.iloc[:, 24]  # Column 24: Can afford favorite cigarette brand?

    # Encode categorical variables
    categorical_features = features.select_dtypes(include=['object']).columns
    features = pd.get_dummies(features, columns=categorical_features, drop_first=True)

    # Encode the target variable (1 for Yes, 0 for No)
    target = target.map({"Yes": 1, "No": 0})

    # Handle missing values
    features.fillna(features.median(), inplace=True)
    target.dropna(inplace=True)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Initialize and train Random Forest
    model = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
    model.fit(X_train, y_train)

    # Feature importance
    feature_importances = pd.DataFrame({
        'Feature': features.columns,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    print("Feature Importance (Correlation Proxy):")
    print(feature_importances)

    return feature_importances

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with your file path

# Analyze feature importance and correlation
if __name__ == "__main__":
    feature_importances = feature_correlation_analysis(file_path)


Feature Importance (Correlation Proxy):
                                              Feature  Importance
9   How would you describe your current income suf...    0.143980
7   To what extent were you financially (negativel...    0.108091
0                        Employment Status_Unemployed    0.079406
1          How often do you feel stressed?_Frequently    0.074936
8   To what extent were you financially (negativel...    0.073031
10  How would you describe your current income suf...    0.069336
13  What is the highest level of education you hav...    0.061584
3        How often do you feel stressed?_Occasionally    0.059492
5   To what extent were you financially (negativel...    0.058659
6   To what extent were you financially (negativel...    0.048909
11  How would you describe your current income suf...    0.038403
15  What is the highest level of education you hav...    0.038019
4              How often do you feel stressed?_Rarely    0.035720
14  What is the highest level of edu

Since the results were sub-optimal we decided to do a weighted model based on the features importance

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

def weighted_random_forest_model(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Select features and target
    features = df[[
        "Employment Status",
        "How often do you feel stressed?",
        "To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?",
        "How would you describe your current income sufficiency?",
        "What is the highest level of education you have attained?"
    ]]
    target = df.iloc[:, 24]  # Column 24: Can afford favorite cigarette brand?

    # Encode categorical variables
    categorical_features = features.select_dtypes(include=['object']).columns
    features = pd.get_dummies(features, columns=categorical_features, drop_first=True)

    # Encode the target variable (1 for Yes, 0 for No)
    target = target.map({"Yes": 1, "No": 0})

    # Handle missing values
    features.fillna(features.median(), inplace=True)
    target.dropna(inplace=True)

    # Calculate feature importances using an initial Random Forest
    temp_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
    temp_model.fit(features, target)
    importances = temp_model.feature_importances_

    # Scale each feature by its importance
    weighted_features = features * importances

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(weighted_features, target, test_size=0.2, random_state=42)

    # Initialize and train the final Random Forest
    model = RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (Yes)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    print("Weighted Random Forest Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with your file path

# Build and train the model
if __name__ == "__main__":
    model = weighted_random_forest_model(file_path)


Weighted Random Forest Model Evaluation:
Accuracy: 0.70
Precision: 0.81
Recall: 0.73
F1 Score: 0.77
ROC-AUC Score: 0.77

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.62      0.55        13
           1       0.81      0.73      0.77        30

    accuracy                           0.70        43
   macro avg       0.66      0.67      0.66        43
weighted avg       0.72      0.70      0.71        43



The results were sub-optimal so we decided to try a different model XGBoost

In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score

def xgboost_model(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Select features and target
    features = df[[
        "Employment Status",
        "How often do you feel stressed?",
        "To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?",
        "How would you describe your current income sufficiency?",
        "What is the highest level of education you have attained?"
    ]]
    target = df.iloc[:, 24]  # Column 24: Can afford favorite cigarette brand?

    # Encode categorical variables
    categorical_features = features.select_dtypes(include=['object']).columns
    features = pd.get_dummies(features, columns=categorical_features, drop_first=True)

    # Encode the target variable (1 for Yes, 0 for No)
    target = target.map({"Yes": 1, "No": 0})

    # Handle missing values
    features.fillna(features.median(), inplace=True)
    target.dropna(inplace=True)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Initialize and train XGBoost Classifier
    model = XGBClassifier(
        random_state=42,
        use_label_encoder=False,  # Suppresses a warning for older XGBoost versions
        eval_metric="logloss",  # Metric to evaluate
        scale_pos_weight=1.5,   # Adjust for class imbalance (tune this value as needed)
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3
    )

    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # Probability of class 1 (Yes)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)

    print("XGBoost Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'  # Replace with your file path

# Build and train the model
if __name__ == "__main__":
    model = xgboost_model(file_path)


XGBoost Model Evaluation:
Accuracy: 0.67
Precision: 0.75
Recall: 0.80
F1 Score: 0.77
ROC-AUC Score: 0.70

Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.38      0.42        13
           1       0.75      0.80      0.77        30

    accuracy                           0.67        43
   macro avg       0.60      0.59      0.60        43
weighted avg       0.66      0.67      0.67        43



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:38:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


The results were similar but a bit better note that we refrained from using the feature in col 25 as there is a direct correlation between it and col 24

# How many cigarettes do you smoke each day?

We'll use the following features to predict it:

Selected Features
*   "Gender:"
*   "How old are you?"
*   "What is your current employment status?"
*   "How often do you feel stressed?"
*   "How would you describe your current income sufficiency

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

def logistic_regression_model(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Define target and features
    target_column = 'How many cigarettes do you smoke each day?'  # Replace with exact target column
    feature_columns = [
        'Gender:',
        'How old are you?',
        'What is your current employment status?',
        'How often do you feel stressed?',
        'How would you describe your current income sufficiency?'
    ]

    # Select features and target
    features = df[feature_columns]
    target = df[target_column]

    # Drop rows with missing target values
    valid_indices = target.dropna().index
    target = target.loc[valid_indices]
    features = features.loc[valid_indices]

    # Preprocess target: Encode categorical target variable
    le_target = LabelEncoder()
    target = le_target.fit_transform(target)

    # Preprocess features: Handle missing values and encode categorical data
    features = features.fillna('Unknown')  # Fill missing values with a placeholder
    features = pd.get_dummies(features, drop_first=True)  # One-hot encoding for categorical features

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Train Logistic Regression
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Logistic Regression Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    return model, le_target

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'
logistic_model, label_encoder = logistic_regression_model(file_path)


Logistic Regression Model Evaluation:
Accuracy: 0.47

Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.62      0.54        16
           1       0.47      0.67      0.55        12
           2       1.00      0.18      0.31        11
           3       0.00      0.00      0.00         4

    accuracy                           0.47        43
   macro avg       0.49      0.37      0.35        43
weighted avg       0.56      0.47      0.43        43



In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

def xgboost_model_with_limitation(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Define target and features
    target_column = 'How many cigarettes do you smoke each day?'  # Replace with exact target column
    feature_columns = [
        'Gender:',
        'How old are you?',
        'What is your current employment status?',
        'How often do you feel stressed?',
        'How would you describe your current income sufficiency?'
    ]

    # Select features and target
    features = df[feature_columns]
    target = df[target_column]

    # Drop rows with missing target values
    valid_indices = target.dropna().index
    target = target.loc[valid_indices]
    features = features.loc[valid_indices]

    # Encode target
    le_target = LabelEncoder()
    target_encoded = le_target.fit_transform(target)

    # Preprocess features: Handle missing values and encode categorical data
    features = features.fillna('Unknown')  # Fill missing values with a placeholder
    features = pd.get_dummies(features, drop_first=True)  # One-hot encoding for categorical features

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

    # Address class imbalance using SMOTE
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train an XGBoost Classifier
    model = XGBClassifier(
        random_state=42,
        use_label_encoder=False,  # Suppresses warning
        eval_metric="mlogloss",  # Multi-class evaluation metric
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3
    )
    model.fit(X_train_resampled, y_train_resampled)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Limit predictions to range 1-40 (if applicable)
    y_pred_limited = [min(max(pred, 1), 40) for pred in y_pred]

    # Decode predictions back to original labels
    y_pred_decoded = le_target.inverse_transform(y_pred_limited)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("XGBoost Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'
xgb_model = xgboost_model_with_limitation(file_path)


XGBoost Model Evaluation:
Accuracy: 0.42

Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.44      0.44        16
           1       0.45      0.42      0.43        12
           2       0.40      0.36      0.38        11
           3       0.33      0.50      0.40         4

    accuracy                           0.42        43
   macro avg       0.41      0.43      0.41        43
weighted avg       0.42      0.42      0.42        43



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:14:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


The results to predict cigarette frequency were pretty difficult to predict even after using a different model and limiting the number of ciggs smoked this may be due to the skewed dataset and the features we chose
# Lets now try predicting stress levels using other features!

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

def predict_stress_frequency(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Define target and features
    target_column = 'How often do you feel stressed?'  # Replace with exact target column
    feature_columns = [
        'Gender:',
        'How old are you?',
        'What is your current employment status?',
        'How would you describe your current income sufficiency?',
        'To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?'
    ]

    # Select features and target
    features = df[feature_columns]
    target = df[target_column]

    # Drop rows with missing target values
    valid_indices = target.dropna().index
    target = target.loc[valid_indices]
    features = features.loc[valid_indices]

    # Encode target
    le_target = LabelEncoder()
    target_encoded = le_target.fit_transform(target)

    # Debug: Print classes in the LabelEncoder
    print("Classes in LabelEncoder:", le_target.classes_)

    # Preprocess features: Handle missing values and encode categorical data
    features = features.fillna('Unknown')  # Fill missing values with a placeholder
    features = pd.get_dummies(features, drop_first=True)  # One-hot encoding for categorical features

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

    # Debug: Print unique classes in train and test sets
    print("Unique classes in training set:", set(y_train))
    print("Unique classes in test set:", set(y_test))

    # Train Logistic Regression
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train, y_train)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Logistic Regression Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, labels=list(set(y_test)), target_names=le_target.classes_))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'
stress_model = predict_stress_frequency(file_path)


Classes in LabelEncoder: ['Constantly' 'Frequently' 'Never' 'Occasionally' 'Rarely']
Unique classes in training set: {0, 1, 2, 3, 4}
Unique classes in test set: {0, 1, 3, 4}
Logistic Regression Model Evaluation:
Accuracy: 0.35

Classification Report:
              precision    recall  f1-score   support

  Constantly       0.00      0.00      0.00         3
  Frequently       0.50      0.50      0.50        22
       Never       0.30      0.25      0.27        12
Occasionally       0.14      0.17      0.15         6

    accuracy                           0.35        43
   macro avg       0.24      0.23      0.23        43
weighted avg       0.36      0.35      0.35        43



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2642: UserWarning: labels size, 4, does not match size of target_names, 5
  warnings.warn(


Not good results lets try a different model


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

def predict_stress_frequency_with_smote(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Define target and features
    target_column = 'How often do you feel stressed?'  # Replace with exact target column
    feature_columns = [
        'Gender:',
        'How old are you?',
        'What is your current employment status?',
        'How would you describe your current income sufficiency?',
        'To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?'
    ]

    # Select features and target
    features = df[feature_columns]
    target = df[target_column]

    # Drop rows with missing target values
    valid_indices = target.dropna().index
    target = target.loc[valid_indices]
    features = features.loc[valid_indices]

    # Combine 'Rarely' and 'Never' classes
    target = target.replace({'Rarely': 'Rarely/Never', 'Never': 'Rarely/Never'})

    # Encode target
    le_target = LabelEncoder()
    target_encoded = le_target.fit_transform(target)

    # Preprocess features: Handle missing values and encode categorical data
    features = features.fillna('Unknown')  # Fill missing values with a placeholder
    features = pd.get_dummies(features, drop_first=True)  # One-hot encoding for categorical features

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

    # Address class imbalance using SMOTE
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Train Random Forest Classifier
    model = RandomForestClassifier(random_state=42, n_estimators=100, class_weight=None)
    model.fit(X_train_resampled, y_train_resampled)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Decode predictions back to original labels
    y_pred_decoded = le_target.inverse_transform(y_pred)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Random Forest Model with SMOTE Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le_target.classes_))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'
rf_smote_model = predict_stress_frequency_with_smote(file_path)


Random Forest Model with SMOTE Evaluation:
Accuracy: 0.40

Classification Report:
              precision    recall  f1-score   support

  Constantly       0.25      0.67      0.36         3
  Frequently       0.53      0.45      0.49        22
Occasionally       0.36      0.33      0.35        12
Rarely/Never       0.20      0.17      0.18         6

    accuracy                           0.40        43
   macro avg       0.33      0.41      0.35        43
weighted avg       0.42      0.40      0.40        43



better but still bad lets try random forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

def predict_stress_frequency_with_rf(file_path):
    # Load the dataset
    df = pd.read_excel(file_path)

    # Define target and features
    target_column = 'How often do you feel stressed?'  # Replace with exact target column
    feature_columns = [
        'Gender:',
        'How old are you?',
        'What is your current employment status?',
        'How would you describe your current income sufficiency?',
        'To what extent were you financially (negatively) affected by the deterioration of the Lebanese economy?'
    ]

    # Select features and target
    features = df[feature_columns]
    target = df[target_column]

    # Drop rows with missing target values
    valid_indices = target.dropna().index
    target = target.loc[valid_indices]
    features = features.loc[valid_indices]

    # Encode target
    le_target = LabelEncoder()
    target_encoded = le_target.fit_transform(target)

    # Preprocess features: Handle missing values and encode categorical data
    features = features.fillna('Unknown')  # Fill missing values with a placeholder
    features = pd.get_dummies(features, drop_first=True)  # One-hot encoding for categorical features

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

    # Train Random Forest Classifier
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Decode predictions back to original labels
    y_pred_decoded = le_target.inverse_transform(y_pred)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print("Random Forest Model Evaluation:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le_target.classes_))

    return model

# Path to the Excel file
file_path = '/content/sample_data/2024_PersonalityTraits_SurveyData.xlsx'
rf_model = predict_stress_frequency_with_rf(file_path)


Random Forest Model Evaluation:
Accuracy: 0.47

Classification Report:
              precision    recall  f1-score   support

  Constantly       0.50      0.67      0.57         3
  Frequently       0.57      0.59      0.58        22
       Never       0.00      0.00      0.00         0
Occasionally       0.38      0.42      0.40        12
      Rarely       0.00      0.00      0.00         6

    accuracy                           0.47        43
   macro avg       0.29      0.33      0.31        43
weighted avg       0.43      0.47      0.45        43



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best so far and we tried to tweak it a bit by  changing weights, features etc. but the accuracy never surpassed 0.55 this may be due to skewed data which is normal in real world data and improper features.

# This is the end of the predictions we tried, note that we used random forest a lot since its the most accurate in these scenarios.

# Thank you!
